In [ ]:
import pandas as pd


In [ ]:
from google.colab import files # Import the 'files' module from the google.colab package. This module is used for file uploading in Google Colab notebooks.
uploaded = files.upload() # Call the upload() function from the 'files' module. This opens a file upload prompt, allowing the user to upload files from their local system to the Colab environment. The uploaded files are stored in the 'uploaded' variable.


Saving msf.csv to msf.csv


In [ ]:
filename = list(uploaded.keys())[0]  # Retrieve the filename of the first (and typically only) uploaded file.

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(filename)  # Load the CSV file into a pandas DataFrame. This DataFrame (df) now contains all the data from your CSV file.

# Define SIC code ranges for utilities and financial firms
utilities_sic_codes = range(4900, 5000)  # Define the range of SIC codes for utilities.
financial_sic_codes = range(6000, 6800)  # Define the range of SIC codes for financial firms.

# Filter out utilities, financial firms, and stocks priced under $5
filtered_df = df[
    ~df['HSICCD'].between(4900, 4999) &  # Exclude utilities: stocks with SIC codes 4900-4999.
    ~df['HSICCD'].between(6000, 6799) &  # Exclude financial firms: stocks with SIC codes 6000-6799.
    (df['PRC'] >= 5)  # Include only stocks with a price of $5 or higher.
]

# Show the first few lines of the filtered DataFrame
print(filtered_df.head())  # Display the first few rows of the filtered DataFrame to verify the filtering process.


        CUSIP  PERMNO  PERMCO  ISSUNO  HEXCD  HSICCD      DATE    BIDLO  \
916  00080010   10006   22156       0      1  3743.0  19251231      NaN   
917  00080010   10006   22156       0      1  3743.0  19260130  109.125   
918  00080010   10006   22156       0      1  3743.0  19260227  101.000   
919  00080010   10006   22156       0      1  3743.0  19260331   95.000   
920  00080010   10006   22156       0      1  3743.0  19260430   93.000   

     ASKHI      PRC  ...        RET      BID     ASK  SHROUT    CFACPR  \
916    NaN  109.000  ...          C  109.375  109.50   600.0  7.412625   
917  113.5  110.250  ...   0.032732  109.500  110.25   600.0  7.260000   
918  110.0  102.375  ...  -0.071429  102.000  102.50   600.0  7.260000   
919  103.0   96.500  ...  -0.042735   96.000   97.00   600.0  7.260000   
920   98.0   94.000  ...  -0.025907   93.500   94.50   600.0  7.260000   

     CFACSHR   ALTPRC  SPREAD    ALTPRCDT       RETX  
916     7.26  109.000     NaN  19251231.0        

In [ ]:
import pandas as pd
import numpy as np

# Convert the 'DATE' column in the dataframe to datetime format for easier manipulation
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

# Convert the 'RET' (returns) column to numeric, coercing any errors (e.g., non-numeric values) to NaN and then forward filling missing returns.
# Forward filling is used to handle missing data by carrying the previous valid return forward.
df['RET'] = pd.to_numeric(df['RET'], errors='coerce').fillna(method='ffill')

# Fill any remaining NaN values in 'RET' with 0, ensuring no disruptions in rolling computations later on
df['RET'] = df['RET'].fillna(0)

# Filter the dataframe to include only rows from the year 1980 onwards, based on the 'DATE' column
df = df[df['DATE'].dt.year >= 1980]

# Sort the DataFrame by 'PERMNO' (unique identifier for each security) and 'DATE' to ensure chronological order for each stock, which is critical for rolling calculations
df = df.sort_values(['PERMNO', 'DATE'])

# Calculate the 12-month rolling average of monthly returns, shifted by one month (using .shift()) to avoid lookahead bias in the analysis
df['rolling_avg_ret'] = df.groupby('PERMNO')['RET'].rolling(window=12, min_periods=1).mean().shift().reset_index(level=0, drop=True)

# Create a new column 'year_month' to identify the year and month of each record, aiding in monthly groupings and sorting
df['year_month'] = df['DATE'].dt.to_period('M')

# Rank stocks into deciles based on their 12-month rolling average return at the end of each month, using the newly created 'year_month' for grouping
df['decile'] = df.groupby('year_month')['rolling_avg_ret'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)

# Create a 'next_month' column to identify the subsequent month, which is useful for calculating future returns
df['next_month'] = df['year_month'].apply(lambda x: x + 1)

# Calculate the equal-weighted returns for each decile and next month combination.
# This is done by grouping by 'year_month' and 'decile', then shifting the returns to align with the following month
df['ew_return'] = df.groupby(['year_month', 'decile'])['RET'].transform('mean').shift(-1)

# Drop columns that are no longer needed ('PERMNO', 'rolling_avg_ret') to simplify the DataFrame
# Also, filter out the last month's data since it does not have a corresponding 'next month's return
df.drop(columns=['PERMNO', 'rolling_avg_ret'], inplace=True)
df = df[df['year_month'] < df['year_month'].max()]

# The DataFrame now contains the equal-weighted returns for each decile for each month.
# As an example, to view the return for the first decile in January 1980:
jan_1980_first_decile_return = df[(df['year_month'] == '1980-01') & (df['decile'] == 1)]['ew_return'].iloc[0]
print(f"Equal-weighted return for the first decile in January 1980: {jan_1980_first_decile_return}")


Equal-weighted return for the first decile in January 1980: -0.004558855445544554


This section of your code focuses on preparing and transforming the data to calculate the 12-month rolling average returns and then ranks these into deciles. It ensures the data is correctly formatted and sorted, which is crucial for accurate calculations in momentum strategy analysis.

In [ ]:
# Calculate the average returns for the first (D1) and tenth (D10) deciles
d1_returns = df[df['decile'] == 1].groupby('year_month')['ew_return'].mean()  # Average returns for the first decile (D1)
d10_returns = df[df['decile'] == 10].groupby('year_month')['ew_return'].mean()  # Average returns for the tenth decile (D10)

# Calculate the returns for the D10-D1 spread (difference between the D10 and D1 decile returns)
d10_d1_returns = d10_returns - d1_returns  # This represents the momentum spread return

# Prepare the portfolio_returns DataFrame to consolidate the results
portfolio_returns = pd.DataFrame({
    'Date': d1_returns.index,  # Dates corresponding to the returns
    'D1': d1_returns.values,  # Returns for the first decile
    'D10': d10_returns.values,  # Returns for the tenth decile
    'D10_D1': d10_d1_returns.values  # Returns for the D10-D1 spread
})

# Convert the 'Date' column to the same format as used in the Fama-French data for consistency
# This is important for any subsequent analysis or merging with other datasets
portfolio_returns['Date'] = portfolio_returns['Date'].dt.to_timestamp().dt.to_period('M')

# The portfolio_returns DataFrame is now ready for CAPM analysis, containing average returns for D1, D10, and the D10-D1 spread, indexed by date

This section of the code focuses on aggregating the returns of specific deciles and computing the difference between the top and bottom deciles. It then organizes these results into a DataFrame that's ready for further CAPM analysis. The conversion of the 'Date' column to match the format of the Fama-French data ensures compatibility for subsequent analysis steps.

In [ ]:
from google.colab import files

# Remove previous versions of the Fama-French data file (if any) from the Colab environment.
# This ensures that only the most recent version of the file is used in the analysis.
# Remove unwanted files from the environment.
!rm -f 'fama_french (1).csv' 'fama_french (2).csv' 'fama_french.csv'

# Prompt the user to upload a new version of the Fama-French data file.
# This is necessary for obtaining the most recent or specific dataset needed for the analysis.
uploaded = files.upload()

# List the files in the current working directory of the Colab environment.
# This command ('ls') is used to confirm that the new file has been successfully uploaded.
!ls

Saving fama_french.csv to fama_french.csv
fama_french.csv  msf.csv  sample_data


In [ ]:
import pandas as pd

# Load the Fama/French 3 Factors dataset. The header is specified to be at index 4 (i.e., the 5th row in the file).
# If the header is not in the 5th row, adjust the 'skiprows' parameter accordingly.
ff3_factors = pd.read_csv('fama_french.csv')

# Convert the 'Date' column from a string to a datetime object and then to a period format.
# This is important for consistency with other datasets (like your portfolio returns dataset) and for ease of time-based analysis.
# The 'format' parameter is set to '%Y%m' to match the date format in the dataset, which typically is YearMonth without a day component.
ff3_factors['Date'] = pd.to_datetime(ff3_factors['Date'], format='%Y%m').dt.to_period('M')

# Check the columns of the loaded DataFrame to ensure that the data is loaded correctly and the 'Date' column is properly formatted.
print(ff3_factors.columns)

# Now convert the percentage format to decimals by dividing by 100.
# This assumes that the Fama-French factors are in percentage format.
# You should verify this with the data documentation or by inspecting the values.
ff3_factors[['Mkt-RF', 'SMB', 'HML', 'RF']] = ff3_factors[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100

# Print the first few rows to confirm the changes.
print(ff3_factors.head())

Index(['Date', 'Mkt-RF', 'SMB', 'HML', 'RF'], dtype='object')
      Date  Mkt-RF     SMB     HML      RF
0  1926-07  0.0296 -0.0256 -0.0243  0.0022
1  1926-08  0.0264 -0.0117  0.0382  0.0025
2  1926-09  0.0036 -0.0140  0.0013  0.0023
3  1926-10 -0.0324 -0.0009  0.0070  0.0032
4  1926-11  0.0253 -0.0010 -0.0051  0.0031


This section of the code focuses on importing the Fama-French 3 Factors data and converting the 'Date' column to a period format. This format conversion is crucial for aligning the Fama-French data with your stock returns data for subsequent analysis, such as CAPM regressions.

In [ ]:
# Display the first few rows (head) of the 'portfolio_returns' DataFrame.
# This is useful to quickly verify the structure and data of the DataFrame, ensuring it contains the expected columns and format.
print(portfolio_returns.head())

# Similarly, display the first few rows of the 'ff3_factors' DataFrame.
# This helps in confirming that the Fama-French 3 Factors data is loaded correctly and is in the expected format.
print(ff3_factors.head())

      Date        D1       D10    D10_D1
0  1980-01 -0.008045 -0.006728  0.001317
1  1980-02 -0.146237 -0.213926 -0.067689
2  1980-03  0.046156  0.046206  0.000050
3  1980-04  0.058356  0.058438  0.000082
4  1980-05  0.025878  0.051933  0.026054
      Date  Mkt-RF     SMB     HML      RF
0  1926-07  0.0296 -0.0256 -0.0243  0.0022
1  1926-08  0.0264 -0.0117  0.0382  0.0025
2  1926-09  0.0036 -0.0140  0.0013  0.0023
3  1926-10 -0.0324 -0.0009  0.0070  0.0032
4  1926-11  0.0253 -0.0010 -0.0051  0.0031


In [ ]:
import pandas as pd
import statsmodels.api as sm  # Import the statsmodels library for statistical modeling.

# Standardize the column names in the Fama-French factors DataFrame for consistency and clarity.
ff3_factors.columns = ['Date', 'Mkt_RF', 'SMB', 'HML', 'RF']

# Merge the portfolio returns DataFrame with the Fama-French factors DataFrame on the 'Date' column.
# This 'inner' merge ensures that only the rows with matching dates in both DataFrames are included in the final merged DataFrame.
df_merged = pd.merge(portfolio_returns, ff3_factors, on='Date', how='inner')

# Calculate the excess returns for the D1, D10, and D10-D1 portfolios by subtracting the risk-free rate ('RF') from each portfolio's returns.
# Excess returns are the returns of an investment above the risk-free rate and are used in CAPM analysis.
df_merged['D1_excess'] = df_merged['D1'] - df_merged['RF']
df_merged['D10_excess'] = df_merged['D10'] - df_merged['RF']
df_merged['D10_D1_excess'] = df_merged['D10_D1'] - df_merged['RF']

# Define the independent variable for the regression (market excess return, 'Mkt_RF') and add a constant term.
# The constant term is necessary for the regression to include an intercept.
X = sm.add_constant(df_merged['Mkt_RF'])

# Initialize an empty DataFrame to store the regression results.
# This DataFrame will include the alpha (intercept), beta (slope), and their respective t-statistics for each portfolio.
results = pd.DataFrame(columns=['Portfolio', 'Alpha', 'Beta', 't-stat Alpha', 't-stat Beta'])

This code prepares your data for CAPM regression analysis by merging the portfolio returns with the Fama-French factors, calculating excess returns, and setting up the independent variable and a DataFrame for the results. The statsmodels library is used for the regression analysis, which will be crucial in understanding the risk and return characteristics of the portfolios.

In [ ]:
# Print the number of rows in the merged DataFrame 'df_merged'.
# This gives an idea of the size of the dataset after merging the portfolio returns with the Fama-French factors.
print(f"Number of rows in merged DataFrame: {df_merged.shape[0]}")

Number of rows in merged DataFrame: 515


This is a useful check to ensure that the merge operation has resulted in a DataFrame of the expected size.

In [ ]:
# Initialize an empty DataFrame for storing the regression results.
# This DataFrame will hold the alpha, beta, and their respective t-statistics for each portfolio.
results = pd.DataFrame(columns=['Portfolio', 'Alpha', 'Beta', 't-stat Alpha', 't-stat Beta'])

# Loop through each portfolio type to perform CAPM regression and gather results.
for portfolio in ['D1_excess', 'D10_excess', 'D10_D1_excess']:
    # Define the dependent variable (Y) as the excess returns of the current portfolio.
    Y = df_merged[portfolio]

    # Perform the Ordinary Least Squares (OLS) regression using the market excess return as the independent variable (X).
    # The OLS method is a common approach in CAPM analysis to estimate the relationship between portfolio and market returns.
    model = sm.OLS(Y, X).fit()

    # Extract the alpha and beta (parameters of the regression) and their respective t-statistics.
    # Alpha represents the portfolio's return in excess of the return predicted by the CAPM, while beta represents its sensitivity to market movements.
    alpha, beta = model.params
    t_alpha, t_beta = model.tvalues

    # Compile the current portfolio's results into a DataFrame.
    # This step includes the portfolio name, alpha, beta, and their t-statistics.
    current_results = pd.DataFrame({
        'Portfolio': [portfolio],
        'Alpha': [alpha],
        'Beta': [beta],
        't-stat Alpha': [t_alpha],
        't-stat Beta': [t_beta]
    })

    # Append the current results to the main results DataFrame.
    # The ignore_index=True parameter ensures that the index is reset appropriately.
    results = pd.concat([results, current_results], ignore_index=True)

# Display the final results table, showing the alpha, beta, and their t-statistics for each portfolio.
print(results)

       Portfolio     Alpha      Beta  t-stat Alpha  t-stat Beta
0      D1_excess -0.019278  0.333079     -5.554508     4.409117
1     D10_excess  0.021748  0.235906      7.492163     3.733731
2  D10_D1_excess  0.037741 -0.092855     15.561024    -1.758904


This section of code is critical for the analysis as it computes the CAPM alpha and beta for each portfolio, providing insights into their performance relative to the market. The loop efficiently handles the regression for each portfolio type, and the results are neatly compiled into a single DataFrame for easy comparison and interpretation.

Alpha (Intercept):

- For D1_excess, the alpha is -0.019278 with a t-statistic of -5.554508. Since the t-statistic is less than -2 (assuming a 95% confidence level), this is statistically significant, indicating that the portfolio underperformed the risk-free rate.

- For D10_excess, the alpha is 0.021748 with a t-statistic of 7.492163. This positive and statistically significant alpha suggests that the portfolio outperformed the risk-free rate.

- For D10_D1_excess, the alpha is 0.037741 with a t-statistic of 15.561024, which is highly significant and indicates strong outperformance relative to the risk-free rate.

Beta (Market Risk):

- For D1_excess, the beta is 0.333079 with a t-statistic of 4.409117. This is significantly different from zero, suggesting that the portfolio has a positive and significant exposure to market risk.

- For D10_excess, the beta is 0.235906 with a t-statistic of 3.733731. Similar to D1_excess, this is also significant, indicating a positive market risk exposure, but to a lesser extent.

- For D10_D1_excess, the beta is -0.092855 with a t-statistic of -1.758904. This t-statistic is close to the threshold of -2 for significance at the 95% confidence level. It suggests that the portfolio might have a negative exposure to market risk, but it's borderline in terms of statistical significance.

In [ ]:
# Print the column names of the DataFrame 'df'.
# This is useful for quickly checking what data columns are available in the DataFrame.
print(df.columns)

Index(['CUSIP', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'DATE', 'BIDLO',
       'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT', 'CFACPR',
       'CFACSHR', 'ALTPRC', 'SPREAD', 'ALTPRCDT', 'RETX', 'year_month',
       'decile', 'next_month', 'ew_return'],
      dtype='object')


This command is especially helpful in data analysis for getting a quick overview of the DataFrame's structure, ensuring that all expected columns are present before proceeding with further analysis or data manipulation.

In [ ]:
# Step 1: Calculate the 12-month rolling standard deviation of returns for each stock.
# This measures the volatility of stock returns over the past year.
df['rolling_std_dev'] = df.groupby('CUSIP')['RET'].rolling(window=12, min_periods=1).std().shift().reset_index(level=0, drop=True)

# Step 2: Rank the stocks each month based on their rolling standard deviation.
# Stocks are ranked in reverse order (lower standard deviation gets a higher rank).
# The 'duplicates' parameter set to 'drop' handles cases where multiple stocks have the same standard deviation.
df['std_dev_decile'] = df.groupby('year_month')['rolling_std_dev'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

# Convert the 'std_dev_decile' to a numerical type and fill NaN values with -1.
# NaN values can occur if there is not enough data to rank some stocks.
df['std_dev_decile'] = pd.to_numeric(df['std_dev_decile'], errors='coerce').fillna(-1)

# Step 3: Create the combined decile ranking (DD) by summing the momentum and volatility deciles.
df['DD'] = df['decile'] + df['std_dev_decile']

# Step 4: Construct the Portfolio based on the combined ranking (DD).
# Select long positions (highest DD values) and short positions (lowest DD values).
long_positions = df[df['DD'].isin([20, 19])]
short_positions = df[df['DD'].isin([2, 3])]

# Calculate the mean returns for long and short positions, shifting by one month to avoid lookahead bias.
long_returns = long_positions.groupby(['year_month'])['RET'].mean().shift(-1)
short_returns = short_positions.groupby(['year_month'])['RET'].mean().shift(-1)

# Calculate the net portfolio returns by subtracting short returns from long returns.
portfolio_returns_DD = long_returns - short_returns

# Prepare the final DataFrame to display the net returns of the portfolio.
final_portfolio_returns_DD = pd.DataFrame({
    'Date': portfolio_returns_DD.index,
    'Net_Return': portfolio_returns_DD.values
})

# Display the first few rows of the final portfolio returns for a quick overview.
print(final_portfolio_returns_DD.head())

      Date  Net_Return
0  1980-01   -0.249611
1  1980-02         NaN
2  1980-03   -0.018192
3  1980-04   -0.037510
4  1980-05   -0.012064


This code effectively combines the concepts of momentum (based on past returns) and volatility (measured by standard deviation) to create a more nuanced investment strategy. It then calculates and displays the returns of a portfolio that takes long positions in stocks with the highest combined ranking and short positions in those with the lowest.

The results here show the net returns of the portfolio for the first few months, starting in 1980. The negative returns in several months suggest that the strategy might not have yielded favorable outcomes initially.



In [ ]:
def perform_capm_regression(portfolio_returns, market_returns):
    # Drop any NaN values from the portfolio returns and check if the series is empty.
    Y = portfolio_returns.dropna()
    if Y.empty:
        return [np.nan] * 4  # Return a list of NaNs if there are no data points to analyze.

    # Align market returns with the portfolio returns dates.
    X = market_returns.loc[Y.index]
    # Add a constant to the market returns to include an intercept in the regression model.
    X = sm.add_constant(X)

    # Perform the Ordinary Least Squares (OLS) regression and fit the model.
    model = sm.OLS(Y, X).fit()
    # Return the alpha (constant term), beta (coefficient for market returns), and their respective t-statistics.
    return model.params['const'], model.params['Mkt_RF'], model.tvalues['const'], model.tvalues['Mkt_RF']

# Align market returns with the dates of the portfolio returns for accurate comparison.
market_returns = df_merged.set_index('Date')['Mkt_RF']

# Calculate the excess returns for the long, short, and the spread (long-short) portfolios.
long_excess_returns = long_returns - df_merged.set_index('Date')['RF']
short_excess_returns = short_returns - df_merged.set_index('Date')['RF']
spread_excess_returns = (long_returns - short_returns) - df_merged.set_index('Date')['RF']

# Perform the CAPM regression for each of the three portfolios (long, short, and spread).
long_alpha, long_beta, long_t_alpha, long_t_beta = perform_capm_regression(long_excess_returns, market_returns)
short_alpha, short_beta, short_t_alpha, short_t_beta = perform_capm_regression(short_excess_returns, market_returns)
spread_alpha, spread_beta, spread_t_alpha, spread_t_beta = perform_capm_regression(spread_excess_returns, market_returns)

# Compile the regression results into a DataFrame for reporting.
results = pd.DataFrame({
    'Portfolio': ['Long', 'Short', 'Spread'],
    'Alpha': [long_alpha, short_alpha, spread_alpha],
    'Beta': [long_beta, short_beta, spread_beta],
    't-stat Alpha': [long_t_alpha, short_t_alpha, spread_t_alpha],
    't-stat Beta': [long_t_beta, short_t_beta, spread_t_beta]
})

# Print the results DataFrame to display the Alpha, Beta, and their respective t-statistics for each portfolio.
print(results)

  Portfolio     Alpha      Beta  t-stat Alpha  t-stat Beta
0      Long  0.008655  0.344117      2.208562     4.037678
1     Short -0.005663  0.189795     -3.778194     5.817206
2    Spread  0.010806  0.160614      3.346756     2.287433


- Long Portfolio:

Alpha: The alpha is 0.008655 with a t-statistic of 2.208562. Since the t-statistic is greater than 2, the alpha is statistically significant at the 95% confidence level, suggesting that the long portfolio has outperformed the risk-free rate after adjusting for market risk.
Beta: The beta is 0.344117 with a t-statistic of 4.037678. This is also statistically significant, indicating a positive and significant exposure to market risk.

- Short Portfolio:

Alpha: The alpha is -0.005663 with a t-statistic of -3.778194. This negative alpha is statistically significant, meaning the short portfolio has underperformed the risk-free rate, which is expected for a short position.
Beta: The beta is 0.189795 with a t-statistic of 5.817206, which is significantly different from zero. The positive beta suggests that the short portfolio has a positive exposure to market risk, which might seem counterintuitive for a short portfolio. This could be due to various factors, such as the types of stocks shorted or market conditions.

- Spread Portfolio:

Alpha: The alpha is 0.010806 with a t-statistic of 3.346756. This positive and statistically significant alpha indicates that the spread strategy (long-short) has yielded returns above the risk-free rate, signifying successful performance.
Beta: The beta is 0.160614 with a t-statistic of 2.287433. The positive beta is marginally statistically significant at the 95% confidence level, suggesting the spread portfolio has some level of market risk exposure, though lower than the long-only portfolio.

**In summary:**

The long portfolio shows significant positive performance above the risk-free rate.
The short portfolio's negative alpha is also significant, indicating underperformance as expected for short positions.
The spread portfolio has a significant positive alpha, suggesting effective performance of the strategy, and its beta indicates some market risk exposure but less than a straightforward long position.

**Comparison and Explanation:**


**Alpha (Performance relative to the risk-free rate):**

- The Long portfolio with momentum (second table) performed better than the Long portfolio without momentum (first table), moving from a significantly negative alpha to a positive one.

- The Short portfolio's performance worsened with the inclusion of momentum, changing from a positive to a negative alpha.

- The Spread portfolio's alpha increased with momentum, suggesting that the strategy benefitted from the inclusion of the momentum factor.

**Beta (Market risk exposure):**

- The Long portfolio's beta remained relatively stable with the inclusion of momentum, suggesting consistent market risk exposure.

- The Short portfolio's beta decreased slightly with momentum, indicating reduced market risk exposure when momentum is considered.

- The Spread portfolio's beta became positive and significant with momentum, in contrast to the negative, borderline significant beta without momentum.

**In summary:**

The results suggest that incorporating momentum into the strategy has a significant impact on performance (alpha) and alters the market risk exposure (beta) to some extent. Specifically:

- The Long momentum portfolio is now outperforming the risk-free rate, whereas the Long portfolio without momentum was underperforming.

- The Short portfolio's performance deteriorates with momentum, underlining the potential impact of momentum in driving short positions.

- The Spread strategy benefits from the inclusion of momentum, achieving better performance with a statistically significant alpha.